In [1]:
from MQGO import data, main, ML_engine, optimizer, QC_engine 
from deepmd.calculator import DP
import subprocess
import numpy as np
import json

2022-04-23 23:17:01.980280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 23:17:01.980412: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
d_steps = [5000]
tot_steps = [10000, 12000]
subprocess.run('cp water1.xyz water.xyz', shell=True)

index = 0
for ii in range(0,len(d_steps)):
    for jj in range(0,len(tot_steps)):
        js_setting = {}
        with open('./input0.json', 'r') as file:
            json_setting = json.load(file)
            print('after remake json: ',json_setting['learning_rate']['decay_steps'])
            
        json_setting['learning_rate']['decay_steps'] = d_steps[ii]
        json_setting['training']['numb_steps'] = tot_steps[jj]
        print('ii,jj:',ii,jj)
        print('before make json: ',json_setting['learning_rate']['decay_steps'])
        with open('./input.json', 'w') as js:
            json.dump(json_setting, js, indent=4)
        
        setting = {'basis':'6-31g', 'spin':0, 'xc':None, 'restricted':True, 'verbose':0}
        geo = main.MLgeomopt(qc_engine='pyscf', ml_engine='deepmd', work_path='./', xyz_path='./water.xyz',**setting)
        geo.opt_algorithm = 'lbfgs'
        geo.debug = True
        geo.kernel()
        
        Opt = optimizer.Optimizer(xyz_path="water0.xyz",
									work_path="./",
									ml_engine='deepmd',
									outter_cycle=1,
									algorithm='lbfgs',
									max_opt_cycle=100)
        atom0 = Opt.ase_read_xyz()
        atom0.calc = DP(model='graph.pb')
        print(atom0.get_potential_energy())
        force0_ml = atom0.get_forces()

        forces = np.loadtxt('raw/force.raw')
        force0_qc = forces[0].reshape((6,3))

        delta = force0_qc-force0_ml
        with open('debug_diff_force.txt', 'a') as file:
            file.write('24.18f\n'%atom0.get_potential_energy())
            for i in range(0, len(force0_qc)):
                for j in range(0, 3):
                    file.write('%24.18f'%delta[i][j])
                file.write('\n')
            file.write('\n')
            
        print('after dp: ',json_setting['learning_rate']['decay_steps'])
        subprocess.run('cp water1.xyz water.xyz', shell=True)
        # subprocess.run('rm input.json', shell=True)

after remake json:  4000
ii,jj: 0 0
before make json:  5000


OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-11
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #216: KMP_AFFINITY: x2APIC ids not unique.
OMP: Info #216: KMP_AFFINITY: decoding legacy APIC ids.
OMP: Info #216: KMP_AFFINITY: legacy APIC ids not unique.
OMP: Info #220: KMP_AFFINITY: parsing /proc/cpuinfo.
OMP: Info #157: KMP_AFFINITY: 12 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #192: KMP_AFFINITY: 1 socket x 6 cores/socket x 2 threads/core (6 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 2 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 3 maps to socket 0 core 1 thread 1 
OMP: Info #172: KMP_AFFINITY: OS p

main.py 81 QC_coords0: [[-0.0311723  0.0234514  1.7547655]
 [ 0.0311723  0.0234514 -1.7547655]
 [ 0.6690259  0.5229127  1.2639675]
 [ 0.0163141 -0.8951651  1.388198 ]
 [-0.0163141 -0.8951651 -1.388198 ]
 [-0.6690259  0.5229127 -1.2639675]]
main.py 82 E_QC0: -4135.179855015961
main.py 83 G_QC0: [[-1.66436201  0.98947387  2.2192323 ]
 [ 1.66436201  0.98947387 -2.2192323 ]
 [ 1.27458198  0.4974947  -1.11036058]
 [ 0.34626678 -1.48696857 -1.03035116]
 [-0.34626678 -1.48696857  1.03035116]
 [-1.27458198  0.4974947   1.11036058]]

			###########################################
			###      Optimization cycle:     1      ###
			###########################################
			


2022-04-23 23:17:06.454304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 23:17:06.454449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
2022-04-23 23:17:11.821531: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 23:17:11.835028: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed cal

optimizer.py 77: [[-0.0311723  0.0234514  1.7547655]
 [ 0.0311723  0.0234514 -1.7547655]
 [ 0.6690259  0.5229127  1.2639675]
 [ 0.0163141 -0.8951651  1.388198 ]
 [-0.0163141 -0.8951651 -1.388198 ]
 [-0.6690259  0.5229127 -1.2639675]]


2022-04-23 23:20:19.428977: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 23:20:19.443893: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-23 23:20:19.444021: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Deng-PC): /proc/driver/nvidia/version does not exist


       Step     Time          Energy          fmax
LBFGS:    0 23:20:20    -4135.179855   2.94515e+00
LBFGS:    1 23:20:20    -4135.587771   2.66501e+00
LBFGS:    2 23:20:20    -4136.925099   1.56984e+00
LBFGS:    3 23:20:20    -4137.627977   9.07867e-01
LBFGS:    4 23:20:20    -4138.007588   5.55859e-01
LBFGS:    5 23:20:20    -4138.237987   5.98320e-01
LBFGS:    6 23:20:20    -4138.415131   7.32429e-01
LBFGS:    7 23:20:20    -4138.592337   9.36608e-01
LBFGS:    8 23:20:20    -4138.412068   7.27207e-01
LBFGS:    9 23:20:20    -4138.224991   5.79647e-01
LBFGS:   10 23:20:20    -4138.419015   7.36810e-01
LBFGS:   11 23:20:20    -4138.625294   9.86152e-01
LBFGS:   12 23:20:20    -4138.416192   7.31595e-01
LBFGS:   13 23:20:20    -4138.213364   5.62518e-01
LBFGS:   14 23:20:20    -4138.422847   7.41064e-01
LBFGS:   15 23:20:20    -4138.657861   1.03729e+00
LBFGS:   16 23:20:20    -4138.420903   7.36780e-01
LBFGS:   17 23:20:20    -4138.205492   5.50361e-01
LBFGS:   18 23:20:20    -4138.4

OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1708 thread 12 bound to OS proc set 0
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1710 thread 13 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1711 thread 14 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1712 thread 15 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1713 thread 16 bound to OS proc set 8
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1714 thread 17 bound to OS proc set 10
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1715 thread 18 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1716 thread 19 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1717 thread 20 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1718 thread 21 bound to OS proc set 7
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1719 thread 22 bound to OS proc set 9
OMP: Info #254: KMP_AFFINITY: pid 1197 tid 1720 thread 23 bound to OS proc set 11


LBFGS:   23 23:20:20    -4138.196846   5.37090e-01
LBFGS:   24 23:20:20    -4137.911702   5.69131e-01
LBFGS:   25 23:20:20    -4138.218145   5.65581e-01
LBFGS:   26 23:20:20    -4138.539192   9.19756e-01
LBFGS:   27 23:20:20    -4138.227345   5.64170e-01
LBFGS:   28 23:20:20    -4137.983275   4.88901e-01
LBFGS:   29 23:20:20    -4138.238864   5.39030e-01
LBFGS:   30 23:20:20    -4138.323083   5.55124e-01
LBFGS:   31 23:20:20    -4138.293819   4.47725e-01
LBFGS:   32 23:20:20    -4138.322629   5.54441e-01
LBFGS:   33 23:20:20    -4138.337922   6.79743e-01
LBFGS:   34 23:20:20    -4138.319311   6.31559e-01
LBFGS:   35 23:20:20    -4138.250713   4.43360e-01
LBFGS:   36 23:20:20    -4138.237103   3.87367e-01
LBFGS:   37 23:20:20    -4138.239145   3.84529e-01
LBFGS:   38 23:20:20    -4138.235650   3.90011e-01
LBFGS:   39 23:20:20    -4138.233976   3.90735e-01
LBFGS:   40 23:20:20    -4138.215010   3.84010e-01
LBFGS:   41 23:20:20    -4138.180294   3.51725e-01
LBFGS:   42 23:20:20    -4138.1